In [1]:
%pip install rasterio
%pip install scikit-learn
%pip install tqdm
%pip install seaborn

          ▄▄▀▀       
       ▄▀▀▀▀▀▀▄▄▄▀▀  
      ▄▀▀▀▀▀▀▀▀▀▀▀▀  
   ▄▄ ▀▀▀▀▀▀▀▀▀▀▀▀   
 ▄▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀    
▄▀▀▀▀▀▀▀▀▀▀▀▀▀       
▀▀▀▀▀▀▀▀▀▀▀▀▀▄ ▄▄ ▄▄▄
 ▀▀ ▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀
     ▀▀▀▀▀▀▀▀▀▀▀▀▀▀  
       ▀▀ ▀▀▀▀▀▀     

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
     ▄▀▀▀▄                     
    ▄▀▀▀▀▀▀▄   ▄▄   ▄▄▀▀▄      
   ▄▀▀▀ ▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▄    
  ▄▀▀▀   ▀▀▀▀▀▀▀▀▀▀▀▀▀   ▀▀▀▄  
▄▀▀▀▀   ▀▀▀▀▀▀▀▀▀▀▀▀▀▄    ▀▀▀▀ 
▀▀▀▀▄   ▀▀ ▀▀▀▀▀▀▀▀▀▀▀     ▀▀▀▀
 ▀▀▀▀▄   ▄▀▀▀▀▀▀▀▀▀▀▀▀    ▄▀▀▀▀
  ▀▀▀▀▀▄▀▀▀▀▀▀▀▀▀▀▀▄▀    ▄▀▀▀▀▀
   ▀▀▀▀▀▀▀▀▀ ▀▀▀▀▀▀▀▀   ▄▀▀▀▀▀ 
   ▀▀▀▀▀▀▀    ▀▀▀ ▀▀▀  ▄▀▀▀▀▀▀ 
     ▀▀               ▀▀▀▀▀▀▀  
                         ▀▀▀   

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
     ▄▀▄    ▄     
     ▀▀▀▀▀▄▀▀▀    


In [2]:
imd_dir = r"/home/stormej/dev/rainscale/data/rain/rain_tif_monthly"
ndvi_dir = r"/home/stormej/dev/rainscale/data/ndvi/ndvi_monthly_resampled_0.25"
lst_dir = r"/home/stormej/dev/rainscale/data/lst/lst_monthly_resampled_0.25"

start_year = 2000
end_year = 2022

In [3]:
import os
import rasterio.transform
from tqdm import tqdm
import rasterio
import numpy as np
from datetime import datetime
from scipy import stats

def extract_date(filename, prefix, date_format):
    date_str = filename.replace(prefix, "").replace(".tif", "")
    try:
        return datetime.strptime(date_str, date_format)
    except ValueError:
        return None

def load_and_process_data(
    rainfall_dir,
    ndvi_dir,
    lst_dir,
    start_year=start_year,
    end_year=end_year,
):
    rainfall_files = sorted(os.listdir(rainfall_dir))
    ndvi_files = sorted(os.listdir(ndvi_dir))
    lst_files = sorted(os.listdir(lst_dir))
    
    data_collection = []
    
    for rf in tqdm(rainfall_files, desc="Processing rainfall files"):
        try:
            rf_date = extract_date(rf, "monthly_rain_", "%Y_%m")
            if rf_date is None or not (start_year <= rf_date.year <= end_year):
                continue
                
            matching_ndvi = next((nf for nf in ndvi_files
                                 if extract_date(nf, "ndvi_resampled_", "%Y-%m") == rf_date), None)
            matching_lst = next((lf for lf in lst_files
                                if extract_date(lf, "lst_resampled_", "%Y-%m") == rf_date), None)
            
            if not (matching_lst and matching_ndvi):
                continue
                
            # Load rainfall data
            with rasterio.open(os.path.join(rainfall_dir, rf)) as rain_src:
                rainfall_data = rain_src.read(1)
                if rainfall_data.size == 0:
                    print(f"Empty rainfall data for {rf}")
                    continue
                    
                rainfall_flat = rainfall_data.flatten()
                rows, cols = np.indices(rainfall_data.shape)
                xs, ys = rasterio.transform.xy(rain_src.transform, rows, cols)
                longitudes = np.array(xs).flatten()
                latitudes = np.array(ys).flatten()
            
            # Load NDVI data
            with rasterio.open(os.path.join(ndvi_dir, matching_ndvi)) as ndvi_src:
                ndvi_data = ndvi_src.read(1)
                if ndvi_data.size == 0:
                    print(f"Empty NDVI data for {matching_ndvi}")
                    continue
                    
                if ndvi_data.shape != rainfall_data.shape:
                    from rasterio.warp import reproject, Resampling
                    ndvi_resampled = np.empty(shape=rainfall_data.shape, dtype=ndvi_data.dtype)
                    reproject(
                        source=ndvi_data,
                        destination=ndvi_resampled,
                        src_transform=ndvi_src.transform,
                        src_crs=ndvi_src.crs,
                        dst_transform=rain_src.transform,
                        dst_crs=rain_src.crs,
                        resampling=Resampling.bilinear
                    )
                    ndvi_flat = ndvi_resampled.flatten()
                else:
                    ndvi_flat = ndvi_data.flatten()
            
            # Load LST data
            with rasterio.open(os.path.join(lst_dir, matching_lst)) as lst_src:
                lst_data = lst_src.read(1)
                if lst_data.size == 0:
                    print(f"Empty LST data for {matching_lst}")
                    continue
                    
                if lst_data.shape != rainfall_data.shape:
                    from rasterio.warp import reproject, Resampling
                    lst_resampled = np.empty(shape=rainfall_data.shape, dtype=lst_data.dtype)
                    reproject(
                        source=lst_data,
                        destination=lst_resampled,
                        src_transform=lst_src.transform,
                        src_crs=lst_src.crs,
                        dst_transform=rain_src.transform,
                        dst_crs=rain_src.crs,
                        resampling=Resampling.bilinear
                    )
                    lst_flat = lst_resampled.flatten()
                else:
                    lst_flat = lst_data.flatten()
            
            # Calculate seasonal features
            month = rf_date.month
            month_sin = np.sin(2 * np.pi * month / 12)
            month_cos = np.cos(2 * np.pi * month / 12)
            month_sin_array = np.full_like(rainfall_flat, month_sin, dtype=np.float64)
            month_cos_array = np.full_like(rainfall_flat, month_cos, dtype=np.float64)
            
            # Check if any array is empty before stacking
            arrays_to_stack = [ndvi_flat, lst_flat, latitudes, longitudes, month_sin_array, month_cos_array]
            if any(arr.size == 0 for arr in arrays_to_stack):
                print(f"Empty array found for {rf}")
                continue
                
            feature_stack = np.column_stack(arrays_to_stack)
            
            mask = (~np.isnan(rainfall_flat)) & (~np.isnan(feature_stack).any(axis=1))
            if not mask.any():
                print(f"No valid data points after masking for {rf}")
                continue
                
            data_collection.append({
                'date': rf_date,
                'features': feature_stack[mask],
                'rainfall': rainfall_flat[mask],
            })
        except Exception as e:
            print(f"Error processing file {rf}: {e}")
            continue
    
    if not data_collection:
        print("Warning: No data was successfully processed. Check your file paths and date formats.")
    return data_collection

def remove_outliers(X, y, n_sigma=3):
    z_scores = np.abs(stats.zscore(X, nan_policy='omit'))
    mask = (z_scores < n_sigma).all(axis=1)
    return X[mask], y[mask]

data_collection = load_and_process_data(
    rainfall_dir=imd_dir,
    ndvi_dir=ndvi_dir,
    lst_dir=lst_dir,
)

Processing rainfall files: 100%|██████████| 276/276 [00:00<00:00, 532.34it/s]


In [4]:
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

X = np.vstack([d['features'] for d in data_collection])
y = np.concatenate([d['rainfall'] for d in data_collection])

print(f"Original data shape - X: {X.shape}, y: {y.shape}")

rainfall_mask = (y >= 0) & (y < np.percentile(y, 99.95))
feature_mask = np.all((X >= np.percentile(X, 0.05, axis=0)) & (X <= np.percentile(X, 99.95, axis=0)), axis=1)

print(f"Rainfall mask: {np.sum(rainfall_mask)} samples remaining")
print(f"Feature mask: {np.sum(feature_mask)} samples remaining")

mask = rainfall_mask & feature_mask
X_cleaned = X[mask]
y_cleaned = y[mask]

print(f"After cleaning - X: {X_cleaned.shape}, y: {y_cleaned.shape}")

if X_cleaned.shape[0] > 0:
    scaler = RobustScaler()
    X_cleaned = scaler.fit_transform(X_cleaned)
else:
    print("ERROR: No data points left after filtering. Adjusting criteria.")
    rainfall_mask = (y >= 0)
    feature_mask = ~np.isnan(X).any(axis=1)
    mask = rainfall_mask & feature_mask
    X_cleaned = X[mask]
    y_cleaned = y[mask]
    print(f"After relaxed cleaning - X: {X_cleaned.shape}, y: {y_cleaned.shape}")
    
    scaler = RobustScaler()
    X_cleaned = scaler.fit_transform(X_cleaned)

split_idx = int(len(X_cleaned) * 0.8)
X_train, X_test = X_cleaned[:split_idx], X_cleaned[split_idx:]
y_train, y_test = y_cleaned[:split_idx], y_cleaned[split_idx:]

param_grid = {
    'n_estimators': [200],
    'max_depth': [5],
    'min_samples_split': [12],
    'min_samples_leaf': [4],
    'max_features': ['sqrt'],
}

rf = RandomForestRegressor(random_state=42, n_jobs=-1)

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

model = RandomForestRegressor(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    max_features=best_params['max_features'],
    random_state=42,
    n_jobs=-1,
    verbose=2,
)

print("Training the model...")

model.fit(X_train, y_train)

y_pred =model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Test MSE: {mse}")
print(f"Test R^2: {r2}")

Original data shape - X: (4125, 6), y: (4125,)
Rainfall mask: 4122 samples remaining
Feature mask: 4113 samples remaining
After cleaning - X: (4110, 6), y: (4110,)
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END max_depth=5, max_features=sqrt, min_samples_leaf=4, min_samples_split=12, n_estimators=200; total time=   0.2s
[CV] END max_depth=5, max_features=sqrt, min_samples_leaf=4, min_samples_split=12, n_estimators=200; total time=   0.2s
[CV] END max_depth=5, max_features=sqrt, min_samples_leaf=4, min_samples_split=12, n_estimators=200; total time=   0.2s
[CV] END max_depth=5, max_features=sqrt, min_samples_leaf=4, min_samples_split=12, n_estimators=200; total time=   0.2s
[CV] END max_depth=5, max_features=sqrt, min_samples_leaf=4, min_samples_split=12, n_estimators=200; total time=   0.2s
Best parameters: {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 12, 'n_estimators': 200}
Training the model...
building tree 11 of 200
buil

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished


### Save Models

In [5]:
import joblib

save_dir = r"/home/stormej/dev/rainscale/data/ml/models"
os.makedirs(save_dir, exist_ok=True)

joblib.dump(model, os.path.join(save_dir, "rainfall_model.joblib"))
joblib.dump(scaler, os.path.join(save_dir, "scaler.joblib"))

['/home/stormej/dev/rainscale/data/ml/models/scaler.joblib']